# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/hannahkollmann/Desktop/Vanderbilt/Homework/Module 6 API Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,36.12,69,20,9.22,CA,1712718615
1,1,bilibino,68.0546,166.4372,18.36,78,100,8.93,RU,1712718696
2,2,vilyuchinsk,52.9306,158.4028,37.92,98,100,2.86,RU,1712718696
3,3,college,64.8569,-147.8028,39.18,45,75,9.22,US,1712718696
4,4,road town,18.4167,-64.6167,77.25,69,20,20.98,VG,1712718623


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    cmap = "Category20",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = .75
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
#I like weather to be no warmer than 90, but at least 60. 
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 60) &
    (city_data_df['Max Temp'] < 90) &
    (city_data_df['Wind Speed'] < 8) &
    (city_data_df['Cloudiness'] == 0)
]
    
# Drop results with null values
ideal_weather_df = ideal_weather_df.dropna().reset_index(drop=True)


# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,23,santa maria,-29.6842,-53.8069,67.24,95,0,1.19,BR,1712718698
1,26,saint-pierre,-21.3393,55.4781,74.88,69,0,5.75,RE,1712718699
2,125,cabo san lucas,22.8909,-109.9124,69.62,63,0,5.75,MX,1712718628
3,150,cartagena,10.3997,-75.5144,82.02,83,0,6.91,CO,1712718678
4,186,taoudenni,22.6783,-3.9836,82.20,12,0,5.64,ML,1712718723
5,215,al hufuf,25.3646,49.5653,73.74,53,0,2.30,SA,1712718727
6,254,mangaratiba,-22.9597,-44.0406,74.61,85,0,3.91,BR,1712718493
7,255,pacific grove,36.6177,-121.9166,60.39,80,0,5.01,US,1712718524
8,257,medina,24.4686,39.6142,66.63,26,0,0.00,SA,1712718733
9,279,sao vicente,-23.9631,-46.3919,80.78,26,0,1.79,BR,1712718737


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,santa maria,BR,-29.6842,-53.8069,95,
1,saint-pierre,RE,-21.3393,55.4781,69,
2,cabo san lucas,MX,22.8909,-109.9124,63,
3,cartagena,CO,10.3997,-75.5144,83,
4,taoudenni,ML,22.6783,-3.9836,12,
5,al hufuf,SA,25.3646,49.5653,53,
6,mangaratiba,BR,-22.9597,-44.0406,85,
7,pacific grove,US,36.6177,-121.9166,80,
8,medina,SA,24.4686,39.6142,26,
9,sao vicente,BR,-23.9631,-46.3919,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel",
     "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat, lng = row["Lat"], row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santa maria - nearest hotel: Hotel Morotin
saint-pierre - nearest hotel: Tropic Hotel
cabo san lucas - nearest hotel: Comfort Rooms
cartagena - nearest hotel: Hotel Bellamar
taoudenni - nearest hotel: No hotel found
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
mangaratiba - nearest hotel: Hotel Portobelo
pacific grove - nearest hotel: Pacific Grove Inn
medina - nearest hotel: The Oberoi Madina
sao vicente - nearest hotel: Pousada Vitória
tan-tan - nearest hotel: Sable d´or
talgram - nearest hotel: No hotel found
jiuquan - nearest hotel: 民族宾馆
mount pleasant - nearest hotel: Old Village Post House
maryland - nearest hotel: Courtyard
catia la mar - nearest hotel: Posada La Parada
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
jeddah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,santa maria,BR,-29.6842,-53.8069,95,Hotel Morotin
1,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
2,cabo san lucas,MX,22.8909,-109.9124,63,Comfort Rooms
3,cartagena,CO,10.3997,-75.5144,83,Hotel Bellamar
4,taoudenni,ML,22.6783,-3.9836,12,No hotel found
5,al hufuf,SA,25.3646,49.5653,53,Al Muhaidab Residence Al Ahsa
6,mangaratiba,BR,-22.9597,-44.0406,85,Hotel Portobelo
7,pacific grove,US,36.6177,-121.9166,80,Pacific Grove Inn
8,medina,SA,24.4686,39.6142,26,The Oberoi Madina
9,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 650,
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)